In [ ]:
!pip install deepxde

In [ ]:
import deepxde as dde
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import time
from google.colab import files

Using backend: tensorflow.compat.v1
Other supported backends: tensorflow, pytorch, jax, paddle.
paddle supports more examples now and is recommended.
Instructions for updating:
non-resource variables are not supported in the long term



In [ ]:

L = 5
pl = 2.5
pl1 = 1
pl2 = 2
pl3 = 3
pl4 = 4

p = 5*10**5
E = 24855578000
I = 0.00045


#convert Newton to MegaNewton
p = p*10**-6
E = E*10**-6

scale = 1

EI = E*I

file_path = '.excel'

func = pd.read_csv(file_path)

x1= func.iloc[:,0].values
y1= func.iloc[:,1].values


# Define the domain

geom = dde.geometry.Interval(0, L)


def dy(x, y):
    return tf.where(x<pl,dde.grad.jacobian(y[:,0:1], x),dde.grad.jacobian(y[:,1:2], x))

def ddy(x, y):
    return tf.where(x<pl,dde.grad.hessian(y[:,0:1], x),dde.grad.hessian(y[:,1:2], x))

def dddy(x, y):
    return tf.where(x<pl,dde.grad.hessian(dde.grad.jacobian(y[:,0:1], x), x),dde.grad.hessian(dde.grad.jacobian(y[:,1:2], x), x))

def pde(x, y):
    y1 = tf.where(x<pl, y[:,0:1], y[:,1:2])
    y2 = tf.where(x>=pl, y[:,1:2], y[:,0:1])
    dy1_xx = dde.grad.hessian(y1, x)
    dy1_xxx = dde.grad.jacobian(dy1_xx, x)
    dy1_xxxx = dde.grad.jacobian(dy1_xxx, x)
    dy2_xx = dde.grad.hessian(y2, x)
    dy2_xxx = dde.grad.jacobian(dy2_xx, x)
    dy2_xxxx = dde.grad.jacobian(dy2_xxx, x)
    return tf.where(x<pl,EI*dy1_xxxx+p,EI*dy2_xxxx+p)

# Boundary conditions

def boundary_l(x, on_boundary):
    return on_boundary and np.isclose(x[0], 0)
def boundary_r(x, on_boundary):
    return on_boundary and np.isclose(x[0], L)

def on_interf(x, _):
    return np.isclose(x[0], pl)

def data_1(x, _):
    return np.isclose(x[0], L/2)
def data_2(x, _):
    return np.isclose(x[0], L/8)
def data_3(x, _):
    return np.isclose(x[0], 7*L/8)
def data_4(x, _):
    return np.isclose(x[0], L/4)
def data_5(x, _):
    return np.isclose(x[0], 3*L/4)
def data_6(x, _):
    return np.isclose(x[0], 3*L/8)
def data_7(x, _):
    return np.isclose(x[0], 5*L/8)

xxxx=np.array(([pl1-0.1],[pl1],[pl1+0.1],[pl2-0.1],[pl2],[pl2+0.1],[pl3-0.1],[pl3],[pl3+0.1],[pl4-0.1],[pl4],[pl4+0.1],[L/2],[L/4],[3*L/4],[L/8],[7*L/8],[3*L/8],[5*L/8],[0],[L],[2*L/20],[18*L/20],[3*L/20],[17*L/20],[2*L/10],[8*L/10],[6*L/10],[4*L/10]))


def continuity_deflection(x, y,_):
    y1 = y[:,0:1]
    y2 = y[:,1:2]
    return [y1,y2]

def continuity_slope(x, y,_):
    y1 = y[:,0:1]
    y2 = y[:,1:2]
    dy1_x = dde.grad.jacobian(y1, x)
    dy2_x = dde.grad.jacobian(y2, x)
    return [dy1_x,dy2_x]

def continuity_moment(x, y, X):
    y1 = y[:,0:1]
    y2 = y[:,1:2]
    dy1_xx = dde.grad.hessian(y1, x)
    dy2_xx = dde.grad.hessian(y2, x)
    return [dy1_xx,dy2_xx]

def continuity_shear(x, y, X):
    y1 = y[:,0:1]
    y2 = y[:,1:2]
    dy1_xx = dde.grad.hessian(y1, x)
    dy1_xxx = dde.grad.jacobian(dy1_xx, x)
    dy2_xx = dde.grad.hessian(y2, x)
    dy2_xxx = dde.grad.jacobian(dy2_xx, x)
    return [dy1_xxx,dy2_xxx]


#Two end fixed
bc1 = dde.icbc.OperatorBC(geom, lambda x, y, _: continuity_deflection(x, y, _)[0], boundary_l)
bc2 = dde.icbc.OperatorBC(geom, lambda x, y, _: EI*continuity_moment(x, y, _)[0], boundary_l)
bc3 = dde.icbc.OperatorBC(geom, lambda x, y, _: continuity_deflection(x, y, _)[1] , boundary_r)
bc4 = dde.icbc.OperatorBC(geom, lambda x, y, _: EI*continuity_moment(x, y, _)[1] , boundary_r)

bc5 = dde.icbc.OperatorBC(geom, lambda x, y, _: continuity_deflection(x, y, _)[0]-continuity_deflection(x, y, _)[1] , on_interf)
bc6 = dde.icbc.OperatorBC(geom, lambda x, y, _: continuity_slope(x, y, _)[0]-continuity_slope(x, y, _)[1] , on_interf)
bc7 = dde.icbc.OperatorBC(geom, lambda x, y, _: EI*continuity_moment(x, y, _)[0]- EI*continuity_moment(x, y, _)[1] , on_interf)
bc8 = dde.icbc.OperatorBC(geom, lambda x, y, _: continuity_deflection(x, y, _)[0] , on_interf)


ic1 = dde.icbc.OperatorBC(geom, lambda x, y, _: continuity_deflection(x, y, _)[1] , data_1) #L/2 ,2.5
ic2 = dde.icbc.OperatorBC(geom, lambda x, y, _: continuity_deflection(x, y, _)[0] + 9.1066*10**-3 , data_2) #L/8 ,0.625
ic3 = dde.icbc.OperatorBC(geom, lambda x, y, _: continuity_deflection(x, y, _)[1] + 9.1066*10**-3 , data_3) #7*L/8 ,4.375
ic4 = dde.icbc.OperatorBC(geom, lambda x, y, _: continuity_deflection(x, y, _)[0] + 1.0751*10**-2 , data_4) #L/4 ,1.25
ic5 = dde.icbc.OperatorBC(geom, lambda x, y, _: continuity_deflection(x, y, _)[1] + 1.0751*10**-2 , data_5) #3*L/4 ,3.75
ic6 = dde.icbc.OperatorBC(geom, lambda x, y, _: continuity_deflection(x, y, _)[0] + 5.7851*10**-3 , data_6) #3*L/8 ,1.875
ic7 = dde.icbc.OperatorBC(geom, lambda x, y, _: continuity_deflection(x, y, _)[1] + 5.7851*10**-3 , data_7) #5*L/8 ,3.125


boundary_condition_list = [bc8,ic1,ic2,ic3,ic4,ic5,ic6,ic7]

# Create empty lists to store results
results = []
losses = []

layer_size = [1] + [20] * 3 + [2]
activation = "tanh"
initializer = "Glorot uniform"
net = dde.nn.FNN(layer_size, activation, initializer)

boundary_conditions = [bc1, bc2, bc3, bc4, bc5, bc6, bc7]

for bc in boundary_condition_list:
    boundary_conditions.append(bc)

    start_time = time.time()
    data = dde.data.PDE(
        geom,
        pde,
        boundary_conditions,
        num_domain=0,
        num_boundary=0,
        #solution=func,
        anchors=xxxx,
    )

    model = dde.Model(data, net)
    model.compile("adam", lr=0.0005)

    # Initialize variables
    iterations_list = [1000, 2000, 2000, 5000, 10000, 20000]

    for iterations in iterations_list:
        losshistory, train_state = model.train(iterations=iterations)

        dde.saveplot(losshistory, train_state, issave=True, isplot=True)
        test_loss = train_state.best_loss_test

        end_time = time.time()
        elapsed_time = end_time - start_time
        print("Elapsed time:", elapsed_time, "seconds")

        N = 49
        x0 = np.linspace(0,L,N)

        X = x0.reshape(-1,1)

        Ys = model.predict(X)

        Y10 = Ys[:,0]
        Y20 = Ys[:,1]

        N1 = round(N * (pl / L))

        Y1 = Y10[:N1]
        Y2 = Y20[N1:]


        Y = np.hstack([Y1,Y2])

        plt.plot(x0, Y)
        plt.xlabel('x')
        plt.ylabel('y')
        plt.title('Plot of y = ODE solution')
        plt.show()


        indices = np.linspace(0, len(x0)-1, 10, dtype=int)  # Get the indices for 10 equally spaced numbers
        #selected_numbers = k[indices]  # Select the numbers at the corresponding indices

        plt.plot(x0, y1)
        plt.xlabel('x')
        plt.ylabel('y')
        plt.title('Function plot')
        plt.show()

        selected_elements_pinn = [Y[i] for i in indices]
        selected_elements_ana = [y1[i] for i in indices]

        differences = [abs(p - a) for p, a in zip(selected_elements_pinn, selected_elements_ana)]  # Calculate differences
        sum_of_differences = sum(differences)/abs(sum(selected_elements_ana))  # Sum all differences

        print("Differences:", differences)
        print("Sum of differences:", sum_of_differences)


        # Store the results in a dictionary
        result = {
            'iterations': iterations,
            'elapsed_time': elapsed_time,
            'sum_of_differences': sum_of_differences,
            'test_loss': test_loss
        }
        results.append(result)
        losses.append(test_loss)

        print(f"Iterations: {iterations}, Elapsed time: {elapsed_time:.2f} seconds")
        print(f"Sum of differences: {sum_of_differences:.2e}")
        print(f"Test loss: {test_loss:.2e}")
        print("")


# Convert results and losses to pandas DataFrames
results_df = pd.DataFrame(results)

# Save the results to an Excel file

with pd.ExcelWriter('/content/results.xlsx') as writer:
    results_df.to_excel(writer, sheet_name='Results_oneload', index=False)


Compiling model...
Building feed-forward neural network...
'build' took 0.084679 s



/usr/local/lib/python3.10/dist-packages/deepxde/nn/tensorflow_compat_v1/fnn.py:116: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  return tf.layers.dense(


In [ ]:
from google.colab import files


# Download the file
files.download('/content/results.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>